In [41]:
# import pyodbc
# connect_string = ''
# try:
#     conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.236.6.111;PORT=1433;DATABASE=Teambuilder;UID=admin;PWD=TeamBuilder;')
#     print("{c} is working".format(c=connect_string))
# except pyodbc.Error as ex:
#     print("{c} is not working".format(c=connect_string))


In [42]:
# import pandas as pd
# ##After connecting to database extract preprocessed information
# #Read in ID and resumes using pandas

# #This is how to create a two column list using two lists newlist = list(zip(numbers,letters))
# #Get data from server
# student_id = []
# student_resume = []
# cursor = conn.cursor()
# cursor.execute("SELECT ALL * FROM StudentInformation") #StudentInformation is the name of the table
# for row in cursor:
#         student_id.append(row.StudentName)
#         student_resume(row.Resume)
# #Store data in pandas data frame
# student_info = list(zip(student_id, student_resume))
# df = pd.DataFrame(student_info, columns=['ID', 'Processed Resume'])
# df.head()

In [43]:
#use the same techniques we used to train to train the model with student names and identites

In [44]:
#Setting Up the Environment
import re #For Preprocessing
import pandas as pd #For Data Handling
from time import time #To Time the Operations
from collections import defaultdict #Use for word frequency

import spacy #More Preprocessing

import logging #Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import multiprocessing

from gensim.models import Word2Vec

In [45]:
from tika import parser
import os
df = pd.DataFrame(columns=['CandidateID', 'resume', 'cleaned_resume'])
for dirname , _, filenames in os.walk('test'):
    for filename in filenames:
       path = os.path.join(dirname, filename)
       raw = parser.from_file(path)
       df = df.append({'CandidateID' : filename[:-4], 'resume' : " ".join(raw['content'].strip().split('\n')[1:]), 'cleaned_resume' : ""}, 
                ignore_index = True)
   
df.shape
df.head()

,CandidateID,resume,cleaned_resume
0,candidate_009,CutThroat Media NLP analyst for user commen...,
1,candidate_046,HOWARD GOODMAN F R E S H E R A N D N L P ...,
2,candidate_070,"Publicis Sapient DATA PROGRAMMER, JAN 2019 -...",
3,candidate_143,M A T T H E W H O R R E S JUNIOR DEVELOPER ...,


In [46]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText

In [47]:
df['cleaned_resume'] = df.resume.apply(lambda x: cleanResume(x))
df = df.drop('resume', 1)
df.shape
df.head()

,CandidateID,cleaned_resume
0,candidate_009,CutThroat Media NLP analyst for user comments...
1,candidate_046,HOWARD GOODMAN F R E S H E R A N D N L P E N ...
2,candidate_070,Publicis Sapient DATA PROGRAMMER JAN 2019 TIL...
3,candidate_143,M A T T H E W H O R R E S JUNIOR DEVELOPER WO...


In [48]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [66]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['cleaned_resume'])
df.head()

,CandidateID,cleaned_resume
0,candidate_009,CutThroat Media NLP analyst for user comments...
1,candidate_046,HOWARD GOODMAN F R E S H E R A N D N L P E N ...
2,candidate_070,Publicis Sapient DATA PROGRAMMER JAN 2019 TIL...
3,candidate_143,M A T T H E W H O R R E S JUNIOR DEVELOPER WO...


In [67]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.0 mins


In [68]:
df_clean = pd.DataFrame(columns=["CandidateID", "clean"])
df_clean = df[["CandidateID"]].copy()
df_clean['clean'] = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean['clean'] = df['CandidateID'] + " " + df_clean['clean']
df_clean.shape
df_clean

,CandidateID,clean
0,candidate_009,candidate_009 cutthroat medium nlp analyst u...
1,candidate_046,candidate_046 howard goodman f r e s h e r n...
2,candidate_070,candidate_070 publicis sapient datum program...
3,candidate_143,candidate_143 m t t h e w h o r r e s junior...


In [69]:
from gensim.models.phrases import Phrases, Phraser

In [70]:
sent = [row.split() for row in df_clean['clean']]

In [71]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

INFO - 16:38:17: collecting all words and their counts
INFO - 16:38:17: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 16:38:17: collected 713 token types (unigram + bigrams) from a corpus of 464 words and 4 sentences
INFO - 16:38:17: merged Phrases<713 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 16:38:17: Phrases lifecycle event {'msg': 'built Phrases<713 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000> in 0.00s', 'datetime': '2021-11-14T16:38:17.958889', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [72]:
bigram = Phraser(phrases)

INFO - 16:38:18: exporting phrases from Phrases<713 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 16:38:18: FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<0 phrases, min_count=30, threshold=10.0> from Phrases<713 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000> in 0.00s', 'datetime': '2021-11-14T16:38:18.002929', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [73]:
sentences = bigram[sent]
print(sentences)

In [74]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

284

In [75]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['datum',
 'work',
 'nlp',
 'learn',
 'machine',
 'python',
 'project',
 'r',
 'e',
 'skill']

In [76]:
import multiprocessing
import gensim
from gensim.models import Word2Vec
from gensim.utils import keep_vocab_item

In [77]:
model = Word2Vec.load('basemodel.bin')
model.min_count = 1
model.alpha = 0.02
model.window = 3

# name = []
# print(len(model.wv.vectors))
# for i in df_clean.CandidateID:
#     for j in range(30):
#         name.append(i)

#     name = bigram[name]
#     model.build_vocab(name, update=True, progress_per=30)
#     model.train(name, total_examples=name.count, epochs=30)
#     name = []
# print(len(model.wv.vectors))

INFO - 16:38:18: loading Word2Vec object from basemodel.bin
INFO - 16:38:18: loading wv recursively from basemodel.bin.wv.* with mmap=None
INFO - 16:38:18: setting ignored attribute cum_table to None
INFO - 16:38:18: Word2Vec lifecycle event {'fname': 'basemodel.bin', 'datetime': '2021-11-14T16:38:18.195104', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}


In [78]:
t = time()
model.window = 3
#rule = gensim.utils.keep_vocab_item('candidate_009', 0, 0, gensim.utils.RULE_KEEP)
model.build_vocab(sentences, update=True, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
print(len(model.wv.vectors))

INFO - 16:38:18: collecting all words and their counts
INFO - 16:38:18: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 16:38:18: collected 284 word types from a corpus of 464 raw words and 4 sentences
INFO - 16:38:18: Updating model with new vocabulary
INFO - 16:38:18: Word2Vec lifecycle event {'msg': 'added 127 new unique words (44.71830985915493%% of original 284) and increased the count of 157 pre-existing words (55.28169014084507%% of original 284)', 'datetime': '2021-11-14T16:38:18.247151', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
INFO - 16:38:18: deleting the raw counts dictionary of 284 items
INFO - 16:38:18: sample=6e-05 downsamples 284 most-common words
INFO - 16:38:18: Word2Vec lifecycle event {'msg': 'downsampling leaves estimated 64.76323488882849 word corpus (14.0%% of prior 464)', 'datetime': '2021-11-14T16:38:18.251154', '

Time to build vocab: 0.0 mins
1400


In [79]:
t = time()

model.train(sentences, total_examples=model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
print(len(model.wv.vectors))

WARNING - 16:38:18: Effective 'alpha' higher than previous training cycles
INFO - 16:38:18: Word2Vec lifecycle event {'msg': 'training model with 23 workers on 1400 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=3 shrink_windows=True', 'datetime': '2021-11-14T16:38:18.305203', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'train'}
INFO - 16:38:18: worker thread finished; awaiting finish of 22 more threads
INFO - 16:38:18: worker thread finished; awaiting finish of 21 more threads
INFO - 16:38:18: worker thread finished; awaiting finish of 20 more threads
INFO - 16:38:18: worker thread finished; awaiting finish of 19 more threads
INFO - 16:38:18: worker thread finished; awaiting finish of 18 more threads
INFO - 16:38:18: worker thread finished; awaiting finish of 17 more threads
INFO - 16:38:18: worker thread finished; awaiting finish of 16 more threads
IN

Time to train the model: 0.01 mins
1400


In [80]:
# KNN Model compare prompt with student
model.wv.most_similar(positive=["candidate_009", "power_system"], negative=["bluetooth"])

[('w', 0.8400382399559021),
 ('decide', 0.8388658761978149),
 ('structured', 0.8382475972175598),
 ('polarisation', 0.8378956913948059),
 ('visualization', 0.8377735018730164),
 ('rajiv', 0.8375554084777832),
 ('nlp', 0.8366607427597046),
 ('gruvil', 0.8366227149963379),
 ('fuel', 0.8365190029144287),
 ('delhi', 0.8362708687782288)]